In [27]:
import pandas as pd
import re
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack

In [12]:
X_train = pd.read_csv('salary-train.csv')
X_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [13]:
X_train['FullDescription'] = X_train['FullDescription'].apply(lambda x: re.sub('[^a-zA-z0-9]', ' ', x.lower()))
X_train.fillna(value='nan', inplace=True)
X_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,nan,20355


In [14]:
enc_tfidf = TfidfVectorizer(min_df=5)
X_train_texts_encoded = enc_tfidf.fit_transform(X_train['FullDescription'])

In [25]:
enc_dic = DictVectorizer()
X_train_cats_encoded = enc_dic.fit_transform(X_train[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [29]:
X_train_conc = hstack((X_train_texts_encoded, X_train_cats_encoded))

In [32]:
clf = Ridge(alpha=1)
clf.fit(X_train_conc, X_train['SalaryNormalized'])

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [33]:
X_test = pd.read_csv('salary-test-mini.csv')
X_test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [34]:
X_test_text_encoded = enc_tfidf.transform(X_test['FullDescription'])
X_test_cats_encoded = enc_dic.transform(X_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [35]:
y_pred = clf.predict(hstack((X_test_text_encoded, X_test_cats_encoded)))
print(y_pred)

[ 56566.56646955  37216.89927848]
